In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from tifffile import imread
from tqdm import tqdm
import pandas as pd
import numpy as np
import tifffile
from scipy import ndimage

base = Path('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/Publication/20240811_Zenodo-Upload/MapMetIP_ProcessedDataset')

In [3]:
samples = list((base / 'img').glob('*'))

In [4]:
panel = pd.read_csv('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/Publication/20240811_Zenodo-Upload/MapMetIP_ProcessedDataset/NB_Panel.csv')
panel = np.array([s.split('_')[0] for s in panel[:len(panel)//2]['name;channel'].to_list()])
panel[38] = 'DNA2'
panel

array(['MPO', 'CD44', 'Fibronectin', 'CD11b', 'HLA-DR', 'PRPH', 'CXCR2',
       'H3K9Ac', 'HLA-ABC', 'CD20', 'LUM', 'CD11c', 'CD24', 'CD3', 'CD45',
       'CD8a', 'GD2', 'CD34', 'CD10', 'CXCR4', 'CD274', 'S100B', 'SOX10',
       'FOXP3', 'CHGA', 'CD279', 'PNMT', 'H4K12Ac', 'GATA3', 'Ki-67',
       'CD56', 'CD4', 'IDO', 'GZMB', 'ELAVL4', 'CD14', 'clPARP', 'DNA1',
       'DNA2', 'Vimentin', 'CD15', 'IF1', 'IF2', 'IF3'], dtype='<U11')

In [5]:
sample_dict = dict()
i=0
for sample in tqdm(samples):
    sample_id = sample.stem
    
    if not 'TU' in sample_id:
        continue
    marker_image_dict = dict()
    
    sample_dict[sample_id] = dict()
    sample_dict[sample_id]['IMAGES'] = tifffile.imread(sample)
    sample_dict[sample_id]['MASKS'] = tifffile.imread(base / f'masks/{sample_id}.TIF').squeeze()
    
    i+=1
    if i > 2:
        continue
        #break

  0%|          | 0/648 [00:00<?, ?it/s]

100%|██████████| 648/648 [01:29<00:00,  7.21it/s]


In [6]:
sample_dict.keys()

dict_keys(['20220929_20220825_03-014_TU_002', '20221018_20220906_03-020_TU_001', '20220927_20220810_05-010_TU_002', '20221019_20220908_21-025_TU_002', '20221019_20220908_20-023_TU_001', '20220929_20220825_03-016_TU_001', '20220925_20220804_18-002_TU_008', '20220928_20220810_04-012_TU_001', '20221020_20220908_20-022_TU_002', '20220927_20220810_05-010_TU_001', '20221018_20220906_03-018_TU_001', '20220927_20220809_08-008_TU_002', '20220925_20220804_18-002_TU_003', '20220927_20220809_08-008_TU_001', '20220927_20220810_05-009_TU_004', '20220927_20220810_05-009_TU_001', '20220928_20220810_04-012_TU_003', '20221018_20220906_03-019_TU_004', '20220929_20220825_03-015_TU_002', '20220926_20220809_16-005_TU_002', '20221020_20220908_20-022_TU_003', '20221019_20220908_20-024_TU_001', '20220929_20220825_03-015_TU_001', '20221018_20220907_13-021_TU_003', '20220926_20220809_10-003_TU_001', '20220925_20220804_17-001_TU_003', '20220926_20220809_10-003_TU_004', '20220928_20220810_04-012_TU_004', '20220928

In [7]:
annotations = pd.read_csv('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/src/MCA/old_imc_metadata.csv')

/tmp/ipykernel_2232/17586222.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations = pd.read_csv('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/src/MCA/old_imc_metadata.csv')


In [8]:
samples = "".join([f'{i}|' for i in sample_dict.keys()])[:-1]
samples = rf'{samples}'

In [9]:
samples

'20220929_20220825_03-014_TU_002|20221018_20220906_03-020_TU_001|20220927_20220810_05-010_TU_002|20221019_20220908_21-025_TU_002|20221019_20220908_20-023_TU_001|20220929_20220825_03-016_TU_001|20220925_20220804_18-002_TU_008|20220928_20220810_04-012_TU_001|20221020_20220908_20-022_TU_002|20220927_20220810_05-010_TU_001|20221018_20220906_03-018_TU_001|20220927_20220809_08-008_TU_002|20220925_20220804_18-002_TU_003|20220927_20220809_08-008_TU_001|20220927_20220810_05-009_TU_004|20220927_20220810_05-009_TU_001|20220928_20220810_04-012_TU_003|20221018_20220906_03-019_TU_004|20220929_20220825_03-015_TU_002|20220926_20220809_16-005_TU_002|20221020_20220908_20-022_TU_003|20221019_20220908_20-024_TU_001|20220929_20220825_03-015_TU_001|20221018_20220907_13-021_TU_003|20220926_20220809_10-003_TU_001|20220925_20220804_17-001_TU_003|20220926_20220809_10-003_TU_004|20220928_20220810_04-012_TU_004|20220928_20220810_04-012_TU_002|20221019_20220908_20-023_TU_003|20220926_20220809_16-006_TU_005|2022092

In [10]:
annotations_ = annotations[annotations["sample_id"].str.contains(samples, na=False)]

In [11]:
from pathlib import Path
imaging_data_folder = Path('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data')
dataset_name = 'IMC_Neuroblastoma'

dataset_folder = imaging_data_folder / dataset_name
dataset_folder.mkdir(exist_ok=True)

target_images_folder = dataset_folder / 'images'
target_images_folder.mkdir(exist_ok=True)

target_masks_folder = dataset_folder / 'masks'
target_masks_folder.mkdir(exist_ok=True)

In [13]:
from collections import Counter

all_annotations = []
for sample, images in sample_dict.items():
    masks = images['MASKS']

    ids = np.unique(masks)
    ids = ids[ids != 0]

    centroids = np.array(ndimage.center_of_mass(masks, labels=masks, index=ids)).round().astype(int)
    mask_values = masks[centroids[:, 0], centroids[:, 1]]
    
    counts = Counter(mask_values)
    valid_mask_values = {val for val, cnt in counts.items() if cnt == 1 and val != 0}
    keep_mask = np.array([val in valid_mask_values for val in mask_values])

    mask_values = mask_values[keep_mask]
    centroids = centroids[keep_mask]
    
    print(len(np.unique(masks))-1, len(np.unique(mask_values)))
    
    sample_indexer = annotations_['sample_id'].str.contains(sample).to_numpy()
    
    sample_annotations = annotations_[annotations_.sample_id == sample]

    sample_replaced = sample.replace('_', '-')

    image_annotations = dict(
        DIM1=[],
        DIM2=[],
        SampleID=[],
        annotation=[],
        mask_value=[],
    ) 
    for cell_id, (DIM1, DIM2) in tqdm(zip(mask_values, centroids)):
        # print(cell_id, DIM1, DIM2)
        curr_cell = sample_annotations[sample_annotations.ObjectNumber == cell_id]
        if len(curr_cell) != 1:
            #print(len(curr_cell))
            continue
        
        image_annotations['DIM1'].append(DIM1)
        image_annotations['DIM2'].append(DIM2)
        image_annotations['SampleID'].append(sample_replaced)
        image_annotations['annotation'].append(curr_cell.celltype.values[0])
        image_annotations['mask_value'].append(cell_id)

    images = images['IMAGES'].transpose(1,2,0)
    
    tifffile.imwrite(target_images_folder / f'{sample_replaced}_image.tif', images)
    tifffile.imwrite(target_masks_folder / f'{sample_replaced}_masks.tif', masks)

    all_annotations.append(pd.DataFrame(image_annotations))
    
    #if len(all_annotations) == 2:
        #break
    
combined_annotations = pd.concat(all_annotations, ignore_index=True)
combined_annotations.to_csv(dataset_folder / 'annotations.csv', index=False)

marker_names = pd.DataFrame(panel, columns=['Label'])
marker_names.to_csv(dataset_folder / 'marker_names.csv')

2753 2753


2753it [00:01, 2645.54it/s]


1469 1467


1467it [00:00, 2641.57it/s]


4338 4338


...

4349it [00:01, 2563.25it/s]


3530 3530


3530it [00:01, 2674.40it/s]


1779 1779


1779it [00:00, 2691.41it/s]


In [12]:
import h5py
import numpy as np

out_path = "/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/src/MCA/dataset/Neuroblastoma_IMC.h5"

with h5py.File(out_path, "w") as f:
    
    str_dtype = h5py.string_dtype(encoding="utf-8")
    f.create_dataset(
        name="MARKERS",
        data=np.array(panel, dtype=str_dtype)
    )
    
    for sample, images_dict in sample_dict.items():
        print(sample, images_dict.keys())
            
        images = images_dict['IMAGES'].transpose(1,2,0)
        segmentation = images_dict['MASKS']
        
        print(images.shape)
        print(segmentation.shape)

        sample = f.create_group(name=str(sample))
        sample.create_dataset(name='IMAGES', data=images, chunks=(64, 64, 1), compression=3)
        sample.create_dataset(name='MASKS', data=images_dict['MASKS'], chunks=(64, 64))
        
    f.create_dataset(name='INDEXER', data=np.array(np.array(indexer_list, dtype=str), dtype=str_dtype))
    f.create_dataset(name='IMAGE_KEYS', data=np.array(list(sample_dict.keys()), dtype=str_dtype))
    f.create_dataset(name='ANNOTATIONS', data=annotations, dtype=str_dtype)
    f.create_dataset(name='META_ANNOTATIONS', data=meta_annotations, dtype=str_dtype)

20220929_20220825_03-014_TU_002 dict_keys(['IMAGES', 'MASKS'])
(696, 696, 44)
(696, 696)
20221018_20220906_03-020_TU_001 dict_keys(['IMAGES', 'MASKS'])
(696, 696, 44)
(696, 696)
20220927_20220810_05-010_TU_002 dict_keys(['IMAGES', 'MASKS'])
(696, 696, 44)
(696, 696)
20221019_20220908_21-025_TU_002 dict_keys(['IMAGES', 'MASKS'])
(696, 696, 44)
(696, 696)
20221019_20220908_20-023_TU_001 dict_keys(['IMAGES', 'MASKS'])
(696, 696, 44)
(696, 696)
20220929_20220825_03-016_TU_001 dict_keys(['IMAGES', 'MASKS'])
(696, 696, 44)
(696, 696)
20220925_20220804_18-002_TU_008 dict_keys(['IMAGES', 'MASKS'])
(696, 696, 44)
(696, 696)
20220928_20220810_04-012_TU_001 dict_keys(['IMAGES', 'MASKS'])
(696, 696, 44)
(696, 696)
20221020_20220908_20-022_TU_002 dict_keys(['IMAGES', 'MASKS'])
(696, 696, 44)
(696, 696)
20220927_20220810_05-010_TU_001 dict_keys(['IMAGES', 'MASKS'])
(696, 696, 44)
(696, 696)
20221018_20220906_03-018_TU_001 dict_keys(['IMAGES', 'MASKS'])
(696, 696, 44)
(696, 696)
20220927_20220809_08-

In [13]:
from torch.utils.data import DataLoader
from MCA.dataset import MultiChannelDataset

ds_paths = [
    "/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/src/MCA/dataset/NB_IMC.h5",
    "/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/src/MCA/dataset/cHL_MIBI.h5"
]

for ds in ds_paths:

    dataset = MultiChannelDataset(
        h5_file_path=ds,
        patch_size=16,
        in_memory=False,
        shuffle=False,
        used_split='all',
        markers_to_use=None, 
        random_state=np.random.randint(0, 1000, 1),
        annotations_key='META_ANNOTATIONS'
    )

    print(dataset)
    print(dataset[0]['img'].shape)

/home/simon_g/anaconda3/envs/MIDL26/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AssertionError: Provided path to h5 file does not exist: /home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/src/MCA/dataset/NB_IMC.h5

In [ ]:
from torch.utils.data import DataLoader
from MCA.dataset import MultiChannelDataset
import numpy as np 

markers_to_use = ['MPO', 'CD44', 'Fibronectin', 'CD11b', 'HLA-DR', 'PRPH', 'CXCR2', 'H3K9Ac', 'HLA-ABC', 'CD20', 'LUM', 'CD11c', 'CD24', 'CD3', 'CD45', 'CD8a', 'GD2', 'CD34', 'CD10', 'CXCR4', 'CD274', 'S100B', 'SOX10', 'FOXP3', 'CHGA', 'CD279', 'PNMT', 'H4K12Ac', 'GATA3', 'Ki-67', 'CD56', 'CD4', 'IDO', 'GZMB', 'ELAVL4', 'CD14', 'clPARP', 'DNA1', 'DNA2', 'Vimentin', 'CD15']

dataset = MultiChannelDataset(
    h5_file_path="/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/src/MCA/dataset/NB_IMC.h5",
    patch_size=16,
    in_memory=False,
    used_split='all',
    markers_to_use=markers_to_use, 
    random_state=np.random.randint(0, 1000, 1),
    annotations_key='ANNOTATIONS',
    mask_image=False,
    shuffle=True
)

dl = DataLoader(dataset, batch_size=32, num_workers=8)

In [ ]:
dataset.channel2idx

In [ ]:
import matplotlib.pyplot as plt
from cellplot_package.cellplot.patches import gridPlot
from copy import copy

expressions, annos = [], []
for batch in tqdm(dl):
    
    #batch['img'][~batch['masks'].bool()] = 0
    #expressions.extend(batch['img'].sum(dim=(1,2)) / batch['masks'].sum(dim=(1,2))[..., None])

    expressions.extend(batch['img'].sum(dim=(1,2)) / batch['img'].shape[1]**2)
    annos.extend(batch['annotation'])
    
    if len(expressions) > 50000:
        break

In [ ]:
expressions = np.array(expressions)
annos = np.array(annos)

In [ ]:
from umap import UMAP
mapper = UMAP()
emb = mapper.fit_transform(expressions)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

annos_mapped = le.fit_transform(annos)

In [ ]:
plt.scatter(emb[:, 0], emb[:, 1], c=annos_mapped, s=1, cmap='tab20')

In [ ]:
np.unique(annos)

In [ ]:
plt.scatter(emb[:, 0], emb[:, 1], c=annos == 'granulocyte', s=0.2, alpha=0.2, vmin=0, vmax=0.5)

In [ ]:
plt.scatter(emb[:, 0], emb[:, 1], c=expressions[:, dataset.channel2idx['CD15']], s=1, alpha=0.2, vmin=0, vmax=0.2)